In [ ]:
# Note: New development should be using Cartopy instead of basemap
# See https://matplotlib.org/basemap/
!sudo apt-get -y install libgeos-dev libgeos++-dev python3-mpltoolkits.basemap

In [ ]:
!sudo pip3 install -U pysal

---
# Visualization using maps in Python

---

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In the Python world, there are multiple tools for data visualizing:
* [**matplotlib**](http://matplotlib.org) produces publication quality figures in a variety of hardcopy formats and interactive environments across platforms; you can generate plots, histograms, power spectra, bar charts, errorcharts, scatterplots, etc., with just a few lines of code;
* [**Seaborn**](http://stanford.edu/~mwaskom/software/seaborn/index.html) is a library for making attractive and informative statistical graphics in Python;
* [**Bokeh**](http://bokeh.pydata.org/en/latest/) targets modern web browsers for presentation; its goal is to provide elegant, concise construction of novel graphics in the style of D3.js, but also deliver this capability with high-performance interactivity over very large or streaming datasets;
* [**plotly**](https://plot.ly) generates the most interactive graphs; allows saving them offline and create very rich web-based visualizations;
* [**folium**](http://folium.readthedocs.org/en/latest) builds on the data wrangling strengths of the Python ecosystem and the mapping strengths of the [Leaflet.js](http://leafletjs.com) JavaScript library;
* [**ggplot**](http://ggplot.yhathq.com) is built for making profressional looking, plots quickly with minimal code;
* [**pygal**](http://www.pygal.org/en/latest) features various graph types like bar charts, line charts, XY charts, pie charts, radar charts, dot charts, pyramid charts, funnel charts, gauge charts; is used for creating svg charts

and others (particularly, pandas also possesses with its own visualization funtionality). Many of above libraries contains various and powerful tools for geovisualization (using maps or globes). We will consider many examples of such visualization kind.

Among all mentioned above libraries only matplotlib is included to [Anaconda](https://www.continuum.io/). The instruction of the library instalation can be found on sites, links to which we have provided above. 

So, before going to the next paragraph, you need install _matplotlib_ (see installation guide [here](http://matplotlib.org/users/installing.html)) together with _basemap_ toolkit for matplotlib (is a library for plotting 2D data on maps in Python) (installation guide can be found [here](http://matplotlib.org/basemap/users/installing.html) and the recommendations for installing of basemap on Windows OS [here](http://stackoverflow.com/questions/18109859/how-to-install-matplotlib-basemap-module-on-windows-7-with-winpython-or-any-pyt)) and _folium_ (also pip using: `pip install folium`).

## Datasets reading and processing data

We’ll be using route, airport, and airline data from [openflights](http://openflights.org/data.html). Each row in the route data corresponds to an airline route between two airports. Each row in the airport data corresponds to an airport in the world, and has information about it. Each row in the airline data represents a single airline. The description and signification of each column you may see [here](http://openflights.org/data.html).

In [ ]:
import pandas as pd

# Create a DataFrame for airports data 
airports = pd.read_csv('/data/airports.csv')
print("Dataset contains",len(airports), "rows.")
airports.head()

In [ ]:
# Create a DataFrame for airlines data
airlines = pd.read_csv('/data/airlines.csv')
print("Dataset contains {} rows.".format(len(airlines.index)))
airlines.head()

In [ ]:
# And a separate DataFrame for routes
routes = pd.read_csv('/data/routes.csv')
# Some rows contain undefined `airline_id`, `source_airport_id` and `destination_airport_id`,
# which are equal to "\\N". Let's drop rows containing these value
routes = routes[(routes['airline_id'] != "\\N") & 
                (routes['source_airport_id'] != "\\N") & 
                (routes['destination_airport_id'] != "\\N")
               ]
# and change its type from `str` to `int`
for col in ['airline_id', 'source_airport_id', 'destination_airport_id']:
    routes[col] = routes[col].astype(int)
print("Dataset contains {} rows.".format(len(routes.index)))
routes.head()

The chosen data is inherently a good fit for mapping – we have latitude and longitude pairs for airports, and for source and destination airports.

The first map we can make is one that shows all the airports all over the world. We can do this with the basemap extension to matplotlib. This enables drawing world maps and adding points, and is very customizable.

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.basemap import Basemap

# This defines the size of the image below
plt.rcParams['figure.figsize'] = (18, 12)

# Create a map on which to draw. 
# 
# First, we select the projection. See https://matplotlib.org/basemap/users/mapsetup.html
# We're using the Gall Stereographic projection https://matplotlib.org/basemap/users/gall.html
# and showing the whole world.
# 
# resolution: the resolution of he included coastlines, lakes, and so on.
# Possible resolutions: c (crude), l (low), i (intermediate), h (high), f (full) 
# 
# area_thresh: the threshold under what no coast line or lake will be drawn.
# lat_0, lon_0: latitude and longitude of map center.
m = Basemap(projection='gall', resolution = 'i', area_thresh = 1000.0, lat_0=0, lon_0=0)
 
# Draw coastlines, borders of countries and the edges of the map.
m.drawcoastlines(linewidth=0.5)
m.drawcountries(linewidth=0.5)
m.fillcontinents(color='#F0F0F0')
m.drawparallels(np.arange(-90.,91.,30.))
m.drawmeridians(np.arange(-180.,181.,60.))
m.drawmapboundary(fill_color='#67C8FF')
m.drawmapboundary()

# Use matplotlib to draw the points onto the map.
x, y = m(list(airports["lon"]), list(airports["lat"]))
m.plot(x, y, 'x', color="red", markersize=1, alpha=0.5)

plt.show()

The map may be more informative if we will manage markers color, shape and size.

In [ ]:
# Define marker color depending on the altitude of airport
def get_marker_color(altitude):
    if altitude <= 200:
        return 'green'
    elif altitude <= 500:
        return 'lightgreen'
    elif altitude <= 1000:
        return 'yellow'
    elif altitude <= 2000:
        return 'orange'
    else:
        return 'red'

# Define marker size and shape based on the amount of outgoing and incoming flights of the airport 
def get_marker_size(airport_id):
    count = routes[(routes['source_airport_id'] == airport_id) | (routes['destination_airport_id'] == airport_id)].shape[0]
    if count < 10:
        shape = 's'   # square
    elif count < 50:
        shape = 'p'   # pentagon
    elif count < 100:
        shape = 'D'   # diamond
    else:
        shape = 'o'   # circle
        
    count = 2 * np.log10(count+1) + 3
    
    return count, shape

# Zoom map on Europe
themap = Basemap(projection='gall',
                 llcrnrlon=-12,              # lower-left corner longitude
                 llcrnrlat=30,               # lower-left corner latitude
                 urcrnrlon=45,               # upper-right corner longitude
                 urcrnrlat=73,               # upper-right corner latitude
                 resolution='i',             # low resolution
                 area_thresh = 1000.0,
                )

themap.drawcoastlines(linewidth=0.5)
themap.drawcountries(linewidth=0.5)
themap.fillcontinents(color='#F0F0F0')
themap.drawmapboundary(fill_color='#67C8FF')
# Draw rivers
themap.drawrivers(color='blue')

# Draw markers with various size and color for respective airport 
for i in airports.index.values:
    x, y = themap(airports.loc[i, "lon"], airports.loc[i, "lat"])
    size, shape = get_marker_size(airports.loc[i, "airport_id"])
    themap.plot(x, y, 
                marker=shape, 
                color=get_marker_color(airports.loc[i, "altitude"]), 
                markersize=size
               )

plt.show()

Adding of text label on the map makes it just more informative. Let's draw top 10 airports by amount of serving airlines 

In [ ]:
import numpy as np
import random

# Find ID of top `n` airports with the largest amount of airlines
def get_top_n_airports(n=10):
    gl = routes.groupby("source_airport_id")["airline_id"].apply(set).apply(len)   # `apply(set)` drops duplicates
    gl.sort_values(ascending=False)
    return gl[:n]


themap = Basemap(projection='robin', resolution = 'i', area_thresh = 1000.0, lat_0=0, lon_0=90)

# Display blue marble image (from http://visibleearth.nasa.gov) as map background
themap.bluemarble(scale=0.5)

# draw lat/lon grid lines every 30 degrees.
themap.drawmeridians(np.arange(0,360,30), linewidth=1, dashes=[4, 2], labels=[1,0,0,1], color='g')
themap.drawparallels(np.arange(-90,90,30), linewidth=1, dashes=[4, 2], labels=[1,0,0,1], color='g')

top = get_top_n_airports()
for _id in top.index.values:
    res = airports[airports["airport_id"] == _id][["lon", "lat", "name"]]
    x, y = themap(res['lon'].values[0], res['lat'].values[0])
    themap.plot(x, y, 'yo', markersize=top[_id] // 5)
    plt.text((1 + 0.02) * x, 
             (1 + random.choice(range(-3,3)) * 0.01) * y, 
             res['name'].values[0], color='r', fontsize=16
            )

plt.show()

It would be pretty cool to see all the air routes on a map. Luckily, we can use basemap to do this. We’ll draw great circles connecting source and destination airports. Each circle will show the route of a single airliner. Unfortunately, there are so many routes that showing them all would be a mess. Instead, we’ll show some random 200 routes.

In [ ]:
themap = Basemap(projection='robin', resolution = 'c', area_thresh = 10000.0, lat_0=0, lon_0=-90)
 
themap.fillcontinents(color='lightgray', lake_color='lightblue')
themap.drawmapboundary(fill_color='lightblue')
themap.drawcountries()
themap.drawcoastlines()

# Iterate through the some random 200 rows
for name, row in routes.sample(200).iterrows():
    try:
        # Get the source and destination airports
        source = airports[airports["airport_id"] == row["source_airport_id"]].iloc[0]
        dest = airports[airports["airport_id"] == row["destination_airport_id"]].iloc[0]
        # Draw a great circle between source and dest airports
        themap.drawgreatcircle(source["lon"], source["lat"], 
                               dest["lon"], dest["lat"],
                               linewidth=1, color='b')
    except (ValueError, IndexError):
        pass

plt.show()

In [ ]:
# And now we may implement our plan about updated map
def get_routes_for_airport(airport_code):
    # Draw the map
    themap = Basemap(projection='robin', resolution = 'c', area_thresh = 10000.0, lat_0=0, lon_0=10)
    themap.fillcontinents(color='lightgray', lake_color='lightblue')
    themap.drawmapboundary(fill_color='lightblue')
    themap.drawcountries()
    themap.drawcoastlines()
    
    # Find the airport
    source = airports[ airports["IATA_FAA"] == airport_code ].iloc[0]
    
    # Find all destination airports where source airport is `source`
    dest_codes = routes[routes.source_airport == airport_code].destination_airport.values
    dest_codes = sorted(set(dest_codes))
    destinations = airports [ airports.IATA_FAA.isin(dest_codes) ]
    
    # Draw a great circle between source and dest airports.
    for index, destination in destinations.iterrows():
        themap.drawgreatcircle(source["lon"], source["lat"],
                               destination["lon"], destination["lat"],
                               linewidth=1, color='b')
    
    display(plt.show())

In [ ]:
get_routes_for_airport('JFK')

Often very informative geovisualization can be reached with the help of globes. `basemap` also provides the possibility of the globe using. Let's build an interactive rotating globe where we random 100 airports with the help of markers. 

In [ ]:
import numpy as np

# Random choice of 100 rows from `airports` DataFrame 
array = []
for i, row in airports.sample(100).iterrows():
    array.append((row['lon'], row['lat']))
    
# Change the size of the image for globe
plt.rcParams['figure.figsize'] = (8, 8)

def globe(lon, lat):
    themap = Basemap(projection='ortho', lon_0=lon, lat_0=lat, resolution='c', area_thresh=10000.0)
    
    themap.drawcoastlines()
    themap.fillcontinents(color='coral', lake_color='lightblue')
    themap.drawparallels(np.arange(-80.,81.,20.))
    themap.drawmeridians(np.arange(-180.,180.,20.))
    themap.drawmapboundary(fill_color='lightblue')
    themap.drawcountries()
    
    # Add markers
    for i in array:
        x, y = themap(i[0], i[1])
        themap.plot(x, y, 'o', color="blue", markersize=8)
    

    clear_output(wait=True)
    display(plt.show())
    
globe(lon=-75, lat=42)

---
## Exercises
---

## Exercise #1: 

> `basemap`'s method `shadedrelief(scale=0.1, origin='lower')` allows drawing maps with Earth relief. Note, attributes in brackets correspond to minimal performance requirements. Find TOP 10 airports wich are situated on the greatest height and TOP 10 airports with the lowest altitude. Display the list of this airports showing its name and country. Also draw the map of Miller projection (`projection="mill"`) using `shadedrelief()` method and show these airports using colored markers. The marker's color depends on the airport's altitude and can be defined with the help of `get_marker_color(altitude)` function which we have used earlier. Marker's size is 10 units. As result, you should get the map like this:

> <img src="images/ex1.png">

## Exercise #2: 

> `routes` DataFrame contains many routes between two airports. Let's caclulate its lengths and draw, for example, 10 the longest of them trajectories on the map (you may choice projection, resolution, etc. yourself). 

>In order to do this, you need to first calculate route lengths. We may use [haversine distance](https://en.wikipedia.org/wiki/Haversine_formula), which calculates the distance between latitude, longitude pairs (in our case, between latitude and longitude values of source and destination airports):
$$d = 2 r \arcsin\left(\sqrt{\sin^2\left(\frac{\varphi_2 - \varphi_1}{2}\right) + \cos(\varphi_1) \cos(\varphi_2)\sin^2\left(\frac{\lambda_2 - \lambda_1}{2}\right)}\right),$$
$\varphi_1$ and $\varphi_2$: latitude of point 1 and latitude of point 2; <br></br>
$\lambda_1$ and $\lambda_2$: longitude of point 1 and longitude of point 2; <br></br>
let's consider the average Earth radius $r$ is equal to 6367 km. 

## Exercise #3: 

> Find TOP 10 countries with the largest and the lowest amounts of airports on their territory and show all airports of these countries using markers on the `folium` map. Circle markers correspond to TOP 10 countries with the largest amount and diamond markers define TOP 10 countries with the lowest amount of airports. Red markers should denote the airports which have no connections with some others, i.e. thay are absent in `routes` DataFrames; blue markers correspond to all others. Marker's size is fixed and equals, for example, to 1000.